In [1]:
%pip install python-dotenv

from dotenv import load_dotenv
import os 
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [2]:
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [3]:

llm = ChatTogether(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0
)

In [4]:
system_message_ta = SystemMessage(
    content="""You are an AI assistant for the Teaching Assistants (TAs) and Admins of the IITM BS Data Science program.
    
    You assist with:
    - Course grading policies and assessment guidelines
    - Managing student inquiries and support requests
    - Academic integrity and plagiarism guidelines
    - Deadlines, resubmissions, and late penalties
    - Handling student complaints and escalations

    Guidelines:
    - Only respond to TA/Admin-related queries.
    - Do NOT provide direct assignment solutions or help with assessments.
    - If a question is beyond your scope, suggest the correct contact or resource.
    - Maintain professional and formal responses.
    - Use retrieved context from official documents when answering.

    Your goal is to support TAs and Admins in managing academic processes efficiently."""
)


In [5]:
prompt_ta = ChatPromptTemplate.from_messages([
    system_message_ta,  
    MessagesPlaceholder(variable_name="retrieved_context"),
    ("human", "{user_query}")
])

autobot_ta = prompt_ta | llm

In [6]:

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# Load TA/Admin Guidelines Document
loader_ta = TextLoader("./assets/TA-Admin-Guidelines.md")  
documents_ta = loader_ta.load()


In [8]:
# Split into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=720, chunk_overlap=72)
chunks_ta = text_splitter.split_documents(documents_ta)


In [9]:
chunks_ta[0].page_content

'## 1. Overview\nThis document outlines the Retrieval-Augmented Generation (RAG) system implemented for the TA/Admin bot in the IITM BS Data Science program.\n\nThe bot assists Teaching Assistants (TAs) and Admins by providing accurate answers to queries related to:\n- Course grading policies and assessment guidelines\n- Managing student inquiries and support requests\n- Academic integrity and plagiarism guidelines\n- Deadlines, resubmissions, and late penalties\n- Handling student complaints and escalations\n\nThe bot retrieves relevant policy documents before generating responses to ensure accuracy.\n\n---\n\n## 2. Architecture'

In [10]:
embedding_model = TogetherEmbeddings()

In [11]:
vector_store_ta = FAISS.from_documents(chunks_ta , embedding_model)

In [12]:

def retrieve_context_ta(user_query):
    
    docs = vector_store_ta.similarity_search(user_query, k=3)
    content =  "\n".join([doc.page_content for doc in docs])
    return [SystemMessage(content=content)]

In [13]:
print(retrieve_context_ta("Can you tell me about the grading of the python course"))


[SystemMessage(content='## Final Course Score Calculation:\nFormula: **Capped at 100**.\n\nWhere:\n- **GAA1:** Average score in the **best 10 out of 11** graded objective assignments.\n- **GAA2:** Average score in the **best 10 out of 11** graded programming assignments.\n- **Qz1:** Quiz 1 score (**0** if not attempted).\n- **PE1:** OPPE1 score (**0** if not attempted).\n- **PE2:** OPPE2 score (**0** if not attempted).\n- **F:** Final exam score.\n\n---\n### Programming Exams (OPPE):\n- **OPPE1:** Sunday, March 2, 2025\n- **OPPE2:** Sunday, April 6, 2025\n\n### OPPE Slot Allocation:\nDepending on eligibility for OPPE1 & OPPE2, students will be allocated one of three slots. Ensure availability on the given dates.\n\n### Eligibility for Bonus:\n- The **SCT (System Compatibility Test)** is mandatory for the bonus to be added to the final score.\n- Attendance in mock tests alone does **not** qualify for the bonus.\n\n## Eligibility Criteria\n\n### OPPE1:\n- Completion of the **OPPE System 

In [14]:
def call_autobot_ta(user_query,retrieved_context):
   return autobot_ta.invoke({"user_query":user_query,"retrieved_context":retrieved_context})

In [15]:
user_query="share a resource for me to add in the course on variables"
retrieved_context = retrieve_context_ta(user_query)
response = call_autobot_ta(user_query,retrieved_context)
print(response.content)

 Here's a resource on variables that you can add to your course:

Title: "Variables in Python"
Link: https://www.w3schools.com/python/python_variables.asp

This resource provides a comprehensive introduction to variables in Python, including:

1. Declaring variables
2. Assigning values to variables
3. Naming conventions
4. Data types (string, integer, float, boolean, list, tuple, dictionary)
5. Mutable and immutable data types
6. Scope of variables
7. Global and local variables
8. Variable assignment and comparison operators
9. Variable lifetime
10. Variable naming best practices

This resource is suitable for beginners and covers the essential concepts of variables in Python, making it a valuable addition to your course.
